In [ ]:
import plotly.express as px
import ast
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
import requests
import os
import nglview as nv

In [16]:
import plotly.graph_objects as go

amino_acid_dict = {
    'A': 'Alanine',
    'C': 'Cysteine',
    'D': 'Aspartic Acid',
    'E': 'Glutamic Acid',
    'F': 'Phenylalanine',
    'G': 'Glycine',
    'H': 'Histidine',
    'I': 'Isoleucine',
    'K': 'Lysine',
    'L': 'Leucine',
    'M': 'Methionine',
    'N': 'Asparagine',
    'P': 'Proline',
    'Q': 'Glutamine',
    'R': 'Arginine',
    'S': 'Serine',
    'T': 'Threonine',
    'V': 'Valine',
    'W': 'Tryptophan',
    'Y': 'Tyrosine'
}

def convert_aa_names(string):
    if string!='Deletion':
        aa1 = string.split(' -> ')[0]
        aa2 = string.split(' -> ')[1]
        return f'{amino_acid_dict[aa1]} -> {amino_acid_dict[aa2]}'
    else:
        return 'Deletion'

In [1]:
! pip uninstall torch -y
! pip install torch
import torch

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
  Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl (150.8 MB)


In [3]:
! pip uninstall transformers -y
! pip install transformers
from transformers import pipeline, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 4.5 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


In [4]:
from src.scripts.mutants_analysis import *

In [8]:
df_mutants = pd.read_csv('data/mutants.csv')
df_merged = pd.read_csv('data/merged_df.csv')
df_mutants['Target Names'] = df_mutants['Target Names'].apply(lambda x: ast.literal_eval(x))
df_mutants['BindingDB Target Chain Sequence'] = df_mutants['BindingDB Target Chain Sequence'].apply(lambda x: ast.literal_eval(x))

In [ ]:
def get_ligand_name_from_smiles(smiles):
    """
    Query ligand name from SMILES on pubchem
    :param smiles: Ligand SMILES 
    :return: ligand name from SMILES as defined on pubchem
    """
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{}/property/IUPACName/JSON".format(smiles)
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data['PropertyTable']['Properties'][0]['IUPACName']
    except Exception as e:
        print(f"Error retrieving name for SMILES {smiles}: {e}")
        return None

In [ ]:
saving_folder_dfs = "src/data/pair_dfs"
saving_folder_ligands = "src/plots/ligands"
ligand_names = []
saving_ligands = False

# Filtering pairs and saving dfs and ligand infos
for index, row in df_mutants.iterrows():
    # Filter to keep only interaction pairs with at least 10 mutants
    if len(row['Target Names']) > 10:
        df_pair = compute_variation_ic50(row, df_merged)
        wt_name = row['UniProt (SwissProt) Entry Name of Target Chain']
        smiles = row['Ligand SMILES']
        ligand_name = get_ligand_name_from_smiles(smiles)
        name = ligand_name + '_' + wt_name
        if df_pair is None:
            # See P2 first cell for explanation of this problem TODO
            print("This pair will not be saved due to multiple conflicting values in BindingDB") 
        else: 
            ligand_names.append(ligand_name)
            # Saving df to csv file for easier access later on
            saving_path_df = saving_folder_dfs + '/' + ligand_name + '_' + '_'.join(wt_name.split(' ')) +'.csv'
            df_pair.to_csv(saving_path_df)
            print("Pair information succesfully saved")
            
            if saving_ligands: # TODO we could remove it right
                # Creating and Saving Ligand Representation from SMILES
                mol = Chem.MolFromSmiles(smiles)

                # Generate 3D coordinates for the molecule
                AllChem.EmbedMolecule(mol, randomSeed=42)
                AllChem.MMFFOptimizeMolecule(mol)

                # Get 3D coordinates from the RDKit molecule
                conf = mol.GetConformer()
                coords = [conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())]

                viewer = py3Dmol.view(width=800, height=600)
                block = Chem.MolToMolBlock(mol)
                viewer.addModel(block, 'mol')

                viewer.setStyle({'stick': {}})
                viewer.setBackgroundColor('white')

                # Saving ligand representation
                saving_path_ligand = os.path.join(saving_folder_ligands, ligand_name)
                viewer.write_html(saving_path_ligand + '.html')

        print("---------------------------------------------------------------------------")

Device set to use cpu


Pair information succesfully saved
---------------------------------------------------------------------------


Device set to use cpu


Pair information succesfully saved
---------------------------------------------------------------------------
For this ligand-protein pair there are multiple values of IC50 and we decided to drop this case.
This pair will not be saved due to multiple conflicting values in BindingDB
---------------------------------------------------------------------------


Device set to use cpu


Pair information succesfully saved
---------------------------------------------------------------------------


Device set to use cpu


Pair information succesfully saved
---------------------------------------------------------------------------


Device set to use cpu


Pair information succesfully saved
---------------------------------------------------------------------------
For this ligand-protein pair there are multiple values of IC50 and we decided to drop this case.
This pair will not be saved due to multiple conflicting values in BindingDB
---------------------------------------------------------------------------


In [ ]:
file_directory = 'src/data/pair_dfs'
saving_directory = 'src/plots'

buttons = []
all_traces = []
total_number_of_mutants_to_display = 0
for file_name in os.listdir(file_directory):
    path = os.path.join(file_directory, file_name)
    df_pair = pd.read_csv(path)

    total_number_of_mutants_to_display += len(df_pair['Mutant Name'].unique())

# Going over every saved interaction df
for idx, file_name in enumerate(os.listdir(file_directory)):
    path = os.path.join(file_directory, file_name)

    df_pair = pd.read_csv(path)
    df_pair.loc[df_pair.Type == 'gap', 'Mutation'] = 'Deletion' # TODO move to creation of df
    df_pair.Mutation = df_pair.Mutation.apply(convert_aa_names) # TODO move to creation of df

    # Add traces for the current file
    visibility = []
    df_pair['Mutant Name'] = df_pair['Mutant Name'].str.replace('cAMP-dependent protein kinase catalytic subunit alpha', 'KAPCA_BOVIN') # TODO move to creation of df
    df_pair['Mutant Name'] = df_pair['Mutant Name'].str.replace('Epidermal growth factor receptor', 'EGFR') # TODO move to creation of df

    # Add every mutant to the plot
    for mutant_name in df_pair['Mutant Name'].unique():
        df_subset = df_pair[df_pair['Mutant Name'] == mutant_name]
        trace = go.Scatter(
            x=df_subset['Positions'],
            y=df_subset['IC50 Ratio'],
            mode='markers',
            name=mutant_name,
            hovertemplate=(
                "<b>Position:</b> %{x}<br>"
                "<b>IC50 Ratio:</b> %{y}<br>"
                "<b>Mutation:</b> %{customdata}<br>"
            ),
            text=df_subset['Mutant Name'],
            customdata=df_subset['Mutation'],
            
            visible=False,
            
        )
        all_traces.append(trace)
        visibility.append(True)

    # Create a button for the current file
    # Determine visibility for the current file
    start_index = len(all_traces)  - len(visibility)
    end_index = len(all_traces)
    file_visibility = [False] * total_number_of_mutants_to_display
    for i in range(start_index, end_index):
        file_visibility[i] = True

    buttons.append({
        'label': f'Interaction pair {idx+1}',
        'method': 'update',
        'args': [
            {'visible': file_visibility}
        ]
    })

    for trace in all_traces:
        trace.visible = False
    for i in range(start_index, end_index):
        all_traces[i].visible = True

# Add all traces to the figure
fig = go.Figure(data=all_traces)

# Add dropdown menu
fig.update_layout(
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'x': 0.7,
        'y': 1.105,
        'showactive': True,
        'active': 4
    }],
    title='Ratio of IC50 values between WT and Mutants by position',
    xaxis_title='Mutation Position',
    yaxis_title='Ratio of IC50 values between WT and a given mutant',
    hovermode='closest'
)

# Show the interactive plot
fig.show()
fig.write_html('test.html')

In [ ]:
# Load the PDB file
def viualize_mutant(protein_file, row=None):
    view = nv.show_file(protein_file)  # Update with the correct file path

    # Remove default representations
    if row.mutations is None:
        print("WT protein")
    else:
        print("Mutant: ", row.mutant_name)
        view.clear_representations()
        view.add_cartoon(color="#D3D3D3")

        for m in row.mutations:
            if m[0] == 'Deletion':
                view.add_cartoon(selection=m[1], color="blue")  # Highlight region 1-18 in blue
            else:
                view.add_ball_and_stick(selection=m[1], color="red")  # Highlight residue 858 in red
    nv.write_html(row.mutant_name + '.html', [view])

test_df = pd.DataFrame({'mutant_name': ['WT', 'mutant [1-18]', 'mutant [C18->T]', 'mutant [1-18][C21->T]'], 'mutations': [None, ['Deletion', '1-18'], ['C18->T', '18'], [['Deletion', '1-18'], ['C21->T', '21']]]})

for name, r in test_df.iterrows():
    viualize_mutant('pdb_files/P00533.pdb', r)

WT protein
Mutant:  mutant [1-18]
Mutant:  mutant [C18->T]
Mutant:  mutant [1-18][C21->T]


In [ ]:
# TBD
from alphafetcher import AlphaFetcher

# Instantiate the fetcher
# The base_savedir parameter allows you to set a base directory where files will be saved.
# Inside this directory, two separate directories for pdb and cif files will be created.
fetcher = AlphaFetcher(base_savedir="")
ids= ['P00533', 'P27791']
# Add desired Uniprot access codes
fetcher.add_proteins(ids)

# Retrieve metadata
fetcher.fetch_metadata(multithread=True, workers=4)
# Metadata available at fetcher.metadata_dict

# Commence download of specified files
fetcher.download_all_files(pdb=True, multithread=True, workers=4)

Fetching files: 100%|██████████| 2/2 [00:00<00:00,  7.91it/s]


In [214]:
# OLD
amino_acid_dict = {
    'A': 'Alanine',
    'C': 'Cysteine',
    'D': 'Aspartic Acid',
    'E': 'Glutamic Acid',
    'F': 'Phenylalanine',
    'G': 'Glycine',
    'H': 'Histidine',
    'I': 'Isoleucine',
    'K': 'Lysine',
    'L': 'Leucine',
    'M': 'Methionine',
    'N': 'Asparagine',
    'P': 'Proline',
    'Q': 'Glutamine',
    'R': 'Arginine',
    'S': 'Serine',
    'T': 'Threonine',
    'V': 'Valine',
    'W': 'Tryptophan',
    'Y': 'Tyrosine'
}

def convert_aa_names(string):
    if string!='Deletion':
        aa1 = string.split(' -> ')[0]
        aa2 = string.split(' -> ')[1]
        return f'{amino_acid_dict[aa1]} -> {amino_acid_dict[aa2]}'
    else:
        return 'Deletion'

file_directory = 'src/data/pair_dfs'
saving_directory = 'src/plots'

for name in os.listdir(file_directory):
    path = os.path.join(file_directory, name)
    df_pair = pd.read_csv(path)
    df_pair.loc[df_pair.Type == 'gap','Mutation'] = 'Deletion'
    df_pair.Mutation = df_pair.Mutation.apply(convert_aa_names)
    # Creating and Saving IC50 plot
    fig = px.scatter(
        df_pair,
        x='Positions',  # x-axis is the mutation position
        y='IC50',  # y-axis is the IC50 value
        color='Mutant Name',  # color by the mutant name
        hover_name='Mutant Name',
        hover_data={'Positions': True, 'Mutation': True, 'IC50': True, 'Mutant Name':False},  # info to display on hover
        title='IC50 Differences of Mutants by Position',
        labels={'Positions': 'Mutation Position', 'IC50': 'IC50 Difference'},
    )

    # Customize layout if needed
    fig.update_layout(
        hovermode='closest',  # Ensure tooltips show up when hovering closest to a point
        xaxis_title='Mutation Position',
        yaxis_title='IC50 Difference',
        template='plotly_dark'  # Optional: use a dark theme (you can remove or change this)
    )

    # Show the interactive plot
    fig.show()
    saving_path = os.path.join(saving_directory, name.split('.')[0])
    fig.write_html(saving_path + '.html')    

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['Unnamed: 0', 'Mutant Name', 'Positions', 'Colour', 'IC50 Difference', 'IC50 Ratio', 'IC50 Percentage', 'Type', 'Mutation', 'Probability'] but received: IC50

In [ ]:
# Probably remove
for name, group in df_mutants[df_mutants['Target Names'].apply(lambda x: len(x)>10)].groupby('WT Target Name'):
    d = {}
    for idx, top_pair in group.iterrows():
        target_names = top_pair['Target Names']
        target_sequences = top_pair['BindingDB Target Chain Sequence']
        d.update(dict(zip(target_names, target_sequences)))

In [ ]:
# Probably remove
def save_to_fasta(group_dict, filename="output.fasta"):
    """
    Saves the protein names and sequences from the dictionary to a FASTA file.
    
    :param group_dict: Dictionary where keys are protein names and values are sequences
    :param filename: Output FASTA file name (default is "output.fasta")
    """
    with open(filename, 'w') as fasta_file:
        for protein_name, sequence in group_dict.items():
            # Write each protein's name and sequence in FASTA format
            fasta_file.write(f">{protein_name}\n")
            # Write the sequence, split into multiple lines if necessary
            for i in range(0, len(sequence), 80):  # 80 characters per line
                fasta_file.write(sequence[i:i+80] + "\n")
    
    print(f"FASTA file saved as {filename}")


In [ ]:
# Probably remove
for name, group in df_mutants[df_mutants['Target Names'].apply(lambda x: len(x)>10)].groupby('WT Target Name'):
    group_dict = {}
    print("-----------------------------------------")
    print('Started processing Group: ', name)
    for idx, top_pair in group.iterrows():
        target_names = top_pair['Target Names']
        target_sequences = top_pair['BindingDB Target Chain Sequence']
        group_dict.update(dict(zip(target_names, target_sequences)))
    
    save_to_fasta(group_dict, f"{name}.fasta")
    print('Finished processing Group: ', name)
    print("-----------------------------------------")

-----------------------------------------
Started processing Group:  Epidermal growth factor receptor
FASTA file saved as Epidermal growth factor receptor.fasta
Finished processing Group:  Epidermal growth factor receptor
-----------------------------------------
-----------------------------------------
Started processing Group:  cAMP-dependent protein kinase catalytic subunit alpha
FASTA file saved as cAMP-dependent protein kinase catalytic subunit alpha.fasta
Finished processing Group:  cAMP-dependent protein kinase catalytic subunit alpha
-----------------------------------------
